<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [297]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error
from sklearn import svm


Ahora, cargamos los datos de prueba en los arreglos `X_train` y `y_train`:

In [303]:
df    = pd.read_csv('demanda.csv')

## Procesamos datos de entrenamiento 
dftrain = df.loc[df.iloc[:,20].isin(['T'])] ## 'T' = training set
y_train = dftrain['Y'].to_numpy()  
X_train = dftrain[['X0','X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']].to_numpy() ## Regresores

## Procesamos datos de prueba 
dftest = df.loc[df.iloc[:,20].isin(['F'])] ## 'F' = test set
y_test = dftest['Y'].to_numpy()
X_test = dftest[['X0','X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']].to_numpy() ## Regresores

In [299]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

In [ ]:
#https://scikit-learn.org/stable/modules/svm.html
#https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html
#https://medium.com/pursuitnotes/support-vector-regression-in-6-steps-with-python-c4569acd062d
mae_svm = []; mse_svm = []; mape_svm = []; Clist = []
for c in range(1, 100, 4):
    Clist.append(c)
    print(c)
    model = svm.SVR(kernel='rbf', C=c)
    model.fit(X_train, y_train)
    #r_sq = model.score(X_test, y_test)
    #print('coefficient of determination:', r_sq)
    
    y_pred = model.predict(X_test)
    mae_svm.append(trunc(mean_absolute_error(y_test,y_pred),4))
    mse_svm.append(trunc(mean_squared_error(y_test,y_pred),4))
    mape_svm.append(trunc(mean_absolute_percentage_error(y_test,y_pred),4))

In [301]:
model = LinearRegression()
model.fit(X_train, y_train)
r_sq = model.score(X_test, y_test)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)
y_pred = model.predict(X_test)
mae_rlm  = trunc(mean_absolute_error(y_test,y_pred),4)
mse_rlm  = trunc(mean_squared_error(y_test,y_pred),4)
mape_rlm = trunc(mean_absolute_percentage_error(y_test,y_pred),4)

coefficient of determination: 0.7459708965464427
intercept: -665.3444836197305
slope: [ 0.          0.28749465  0.29204618  0.25861088 -0.04431544 -0.14208937
 -0.02613895  0.18289727  0.33580077  0.06905434 -0.12973508]


In [302]:
print('| REGRESIÓN      | C              | MAE            | MSD            | MAPE         |')
print('| :------------- | :------------- | -------------: | -------------: |-------------:|')
print('|    lineal      |         ',       0,'    |    ',str(mae_rlm),'  |    ',str(mse_rlm),'    |    ',str(mape_rlm),'|' )
for i in range(len(Clist)):
    print('|     SVM        |       ',  Clist[i]  ,'    |    ',str(mae_svm[i]),'   |    ',str(mse_svm[i]),'    |    ',str(mape_svm[i]),'|')


| REGRESIÓN      | C              | MAE            | MSD            | MAPE         |
| :------------- | :------------- | -------------: | -------------: |-------------:|
|    lineal      |          0     |     167.1343   |     48064.1398     |     0.0192 |
|     SVM        |        1     |     359.7775    |     181254.7772     |     0.0411 |
|     SVM        |        5     |     320.6393    |     145641.187     |     0.0366 |
|     SVM        |        9     |     284.6851    |     117376.277     |     0.0323 |
|     SVM        |        13     |     252.8958    |     94847.4314     |     0.0286 |
|     SVM        |        17     |     223.7569    |     75391.666     |     0.0252 |
|     SVM        |        21     |     203.931    |     62997.5934     |     0.023 |
|     SVM        |        25     |     185.3293    |     52599.537     |     0.0209 |
|     SVM        |        29     |     170.0756    |     45447.9205     |     0.0191 |
|     SVM        |        33     |     160.565    |  

https://towardsdatascience.com/an-introduction-to-support-vector-regression-svr-a3ebc1672c2